In [1]:
!pip install allantools

  Created wheel for allantools: filename=AllanTools-2019.9-py2.py3-none-any.whl size=40677 sha256=931a37df64b20a09370c4348472db3528a26f388b8f0881d9c80fbdf4fb2ce2a
  Stored in directory: /root/.cache/pip/wheels/4f/2d/a2/2818a818596ce0cbb0a89732710d77224ec6f46fafc4122eb1
Successfully built allantools


In [2]:
!python -m pip install "dask[complete]"

     |████████████████████████████████| 112kB 5.0MB/s 
     |████████████████████████████████| 706kB 7.5MB/s 
ERROR: distributed 2021.4.1 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
ERROR: distributed 2021.4.1 has requirement dask>=2021.03.0, but you'll have dask 2.12.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [3]:
import allantools
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

In [4]:
import dask.dataframe as dd

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
#reading csv directly should be less memory expensive than reading a loaded pandas dataframe
df_dask = dd.read_csv("/content/drive/MyDrive/Colab Notebooks/IMU Error Research/Datasets/combined_IMU_data.csv")

In [7]:
#default memory usage
#sensor = dfd["Sensor"].to_dask_array(lengths=True).astype("int64")
#sensor.nbytes/1e6 #1086.310248

In [8]:
#only able to use i16 because know that largest value < max(int16)
#sensor = dfd["Sensor"].to_dask_array(lengths=True).astype("int16")
#sensor.nbytes/1e6 #271.577562

In [9]:
#use int16 instead of default int64
sensor = df_dask["Sensor"].to_dask_array(lengths=True).astype("int16")
laser = df_dask["Laser"].to_dask_array(lengths=True).astype("int16")

In [10]:
def log_straight_line(intercept, slope, x):
    """
    Create an array of values that follows:
    log(y) = (slope)*log(x) + intercept

    on log-log scale

    This is also the power law:
    y = (intercept)*(x**slope)

    Args:
    intercept (int or float):  The y-intercept of the loglog straight line, or
                                the coefficient of the power law.
    slope (float):  The slope of the loglog straight line, or the power law
                    exponent.
    x (array):  Inputs to the function.

    Returns:
    (array):  Outputs of the power law. 
    """
    return intercept*(np.power(x, slope))

In [11]:
def calculate_adev(series, title, filename, ref={"intercept":10, "slope":-0.5, "x":np.arange(1, 100)}):

    """
    Calculate the Allan Deviation of a time series.
    Save the figure in the local directory.

    Args:
    series (DataFrame, or array-like):  Time series values "y(t)"
    title (str):  Title of the figure
    filename (str):  Name of the saved figure file. Must include file extension
                        in string.
    ref (dict):  Dictoinary with parameters for plotting a reference line.
                        Defaults to {"intercept":10,
                                    "slope":-0.5,
                                    "x":np.arange(1, 100)}
    
    """

    t, a, _, n = allantools.adev(series, rate=1/10e-6)
    
    #take the square root of the variance
    a = np.sqrt(a)

    #create figure to plot
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)

    #place ADEV on axes
    adev, = ax.plot(t, a, label="Allan Deviation")

    #calculate reference line   
    y = log_straight_line(ref["intercept"], ref["slope"], ref["x"])
    #place reference line on axes
    line, = ax.plot(ref["x"], y, lw=4.0, label="Reference Slope {}".format(ref["slope"]))
    
    ax.set_yscale("log")
    ax.set_xscale("log")

    ax.set_xlabel("$\\tau$")
    ax.set_xlabel("$\sigma(\\tau)$")
    ax.set_title(title)
    ax.legend()
    ax.grid(b=True)

    plt.savefig(filename)
    plt.close()

    return 

In [12]:
calculate_adev(sensor, "ADEV of Sensor Data (Lia 2017)", "sensor_adev.png")

In [13]:
calculate_adev(laser, "ADEV of Laser Data (Lia 2017)", "laser_adev.png")